In [12]:
from pathlib import Path

In [13]:
from smtag.config import config
config.from_pretrained

'roberta-base'

In [14]:
from transformers import __version__
__version__

'4.15.0'

## Extracting examples for LM

In [4]:
from smtag.extract import ExtractorXML

In [8]:
corpus = "/data/xml/mini"
text_examples = "/data/text/mini"
tokenized_examples = "/data/json/mini"
xpath = ".//abstract"
sentence_level = False
keep_xml = False
inclusion_probability = 1.0

In [6]:
extractor_lm = ExtractorXML(
    corpus,
    destination_dir=text_examples,
    sentence_level=sentence_level,
    xpath=xpath,
    keep_xml=keep_xml,
    inclusion_probability=inclusion_probability
)

/data/text/mini created


In [7]:
extractor_lm.extract_from_corpus()

100%|██████████| 1/1 [00:00<00:00, 46.48it/s]


{PosixPath('/data/text/mini/train.txt'): 6,
 PosixPath('/data/text/mini/eval.txt'): 2,
 PosixPath('/data/text/mini/test.txt'): 2}

## Preparing tokenized dataset for LM

In [10]:
from smtag.dataprep import PreparatorLM

In [11]:
prep_lm = PreparatorLM(
    text_examples,
    tokenized_examples,
    max_length=config.max_length
)

ValueError: /data/json/mini already exists! Will not overwrite pre-existing dataset.

In [10]:
prep_lm.run()

Preparing: train


100%|██████████| 6/6 [00:00<00:00, 155.79it/s]


Length verification: OK!

average input_ids length = 353 (min=168, max=391) tokens



2022-02-04 22:52:51.896751: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-04 22:52:51.896806: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


longest example: <s>Expression of developmentally regulated membrane proteins of aggregating cells of Dictyostelium discoideum is subject to several control mechanisms. One of them involves periodic cyclic-AMP pulses as signals for gene expression. To increase the probability of selecting mutants specifically defective in the contact site A (csA) glycoprotein, one of the characteristic proteins of aggregating cells, we have bypassed the requirement for both cyclic-AMP pulses and another control element by two runs of mutagenesis. A `double bypass' mutant, HG592, was obtained which aggregated in nutrient medium where wild-type did not develop. Mutants defective in expression of the csA-glycoprotein were selected from HG592 by fluorescence-activated cell sorting and colony immunoblotting using a monoclonal antibody specific for that protein. One among 51 csA-negative mutants, HG693, specifically lacked the capability of forming EDTA-stable intercellular contacts. It acquired chemotactic 

100%|██████████| 2/2 [00:00<00:00, 64.57it/s]



Length verification: OK!

average input_ids length = 449 (min=215, max=234) tokens
longest example: <s>We report the chemical characterization of the highly sulphated glycoprotein SSG 185 from Volvox carteri. SSG 185 is a hydroxyproline-containing, extracellular glycoprotein. The sulphate residues are clustered within the parent saccharide structure of SSG 185, since on mercaptolysis all the sulphate residues are recovered in a small saccharide fragment containing mannose, arabinose and sulphate (in a molar ratio of 1̇1̇2). SSG 185 is a short-lived molecule, serving as a precursor for a high mol. wt. component of the extracellular matrix. Synthesis of SSG 185 is developmentally controlled. Different SSG 185 variants, with unknown modifications in the sulphated saccharide fragment, are synthesized at different developmental stages or under the influence of the sexual inducer. These modifications remain conserved in the aggregated state of SSG 185, indicating the development-dependent m

100%|██████████| 2/2 [00:00<00:00, 73.77it/s]


Length verification: OK!

average input_ids length = 417 (min=156, max=261) tokens
longest example: <s>Two contrasting mechanisms have been proposed for the establishment of the prestalk-prespore pattern in the multicellular aggregate of the simple eukaryote Dictyostelium discoideum. One involves intermingled, non-position-dependent cell differentiation followed by sorting out which produces the pattern of prestalk cells in the anterior region and prespore cells posteriorly. The second mechanism involves patterning according to the position of cells within the aggregate, in which case intermingled cell types are not expected. Here we use a monoclonal antibody (MUD1), recognising a prespore cell surface antigen, to study the initial appearance of prespore cells in aggregates. Quantitative studies were made with a flow cytometer and frozen sections were used to localise the cells expressing the prespore antigen. This antigen first appeared at the onset of tip formation in the centre of 

## Train LM

In [4]:
from smtag.train.train_lm import (
    train as train_lm,
    TrainingArgumentsLM
)

2022-02-04 22:54:30.629472: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-04 22:54:30.629526: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
no_cache = False
path = "./smtag/loader/loader_lm.py"  # includes a loading script with the same name ./lm/lm.py
data_config_name = "MLM"
tokenizer = config.tokenizer  # tokenizer has to be the same application-wide

In [6]:
training_args_tokcl = TrainingArgumentsLM(
    num_train_epochs = 1,
    logging_steps = 5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
)
training_args_tokcl

TrainingArgumentsLM(output_dir='/lm_models', overwrite_output_dir=True, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=True, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, log_level=-1, log_level_replica=-1, log_on_each_node=True, logging_dir='/lm_models/runs/Feb04_22-54-41_7ae45ce523d0', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=5, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=500, save_total_limit=5, save_on_each_node=False, no_cuda=False, seed=42, bf16=Fa

In [9]:
train_lm(
    no_cache,
    path,
    tokenized_examples,
    data_config_name,
    training_args_tokcl,
    tokenizer
)

tokenizer vocab size: 50265

Loading datasets found in /data/json/mini.
using ./smtag/loader/loader_lm.py as dataset loader.


  0%|          | 0/3 [00:00<?, ?it/s]


Training with 6 examples.
Evaluating on 2 examples.


***** Running training *****
  Num examples = 6
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 6



Training arguments:
TrainingArgumentsLM(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=5,
evaluation_strategy=IntervalStrategy.STEPS,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=

...

***** Running Evaluation *****
  Num examples = 2
  Batch size = 1




<s>We report the chemical characterization of the highly dissemin[sulph]ated glycoprotein SSG 185 from Volvaria[ox] carteri. SSG 185 is a hydroxyproline-containing, extracellular glycoprotein. The sulphate residues are clustered in[within] the first[parent] saccharide residues[structure] of SSG 185, since</s>






Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /lm_models
Configuration saved in /lm_models/config.json
Model weights saved in /lm_models/pytorch_model.bin
***** Running Prediction *****
  Num examples = 2
  Batch size = 1


Testing on 2.


ValueError: Found input variables with inconsistent numbers of samples: [15, 753975]